In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [2]:
from PIL import Image 

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [4]:
train_path = r"C:\Users\joe\Desktop\College\Final Project\Final Year Project\train"
test_path = r"C:\Users\joe\Desktop\College\Final Project\Final Year Project\test"

In [5]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [6]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [7]:
  # useful for getting number of classes
folders = glob(r"C:\Users\joe\Desktop\College\Final Project\Final Year Project\train\*")
print(len(folders))

3


In [8]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [10]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [11]:
# view the structure of the model
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [12]:
from keras import optimizers


adam = optimizers.Adam()
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [13]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [14]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [15]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2235 images belonging to 3 classes.


In [16]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 559 images belonging to 3 classes.


In [17]:
from PIL import Image 
from datetime import datetime
from keras.callbacks import ModelCheckpoint



checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=2, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model_history=model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set),
    callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10

Epoch 00001: val_loss improved from inf to 0.94849, saving model to mymodel.h5
70/70 - 632s - loss: 1.1961 - accuracy: 0.8649 - val_loss: 0.9485 - val_accuracy: 0.8945
Epoch 2/10

Epoch 00002: val_loss improved from 0.94849 to 0.57687, saving model to mymodel.h5
70/70 - 636s - loss: 0.5444 - accuracy: 0.9351 - val_loss: 0.5769 - val_accuracy: 0.9374
Epoch 3/10

Epoch 00003: val_loss improved from 0.57687 to 0.21336, saving model to mymodel.h5
70/70 - 633s - loss: 0.3360 - accuracy: 0.9606 - val_loss: 0.2134 - val_accuracy: 0.9732
Epoch 4/10

Epoch 00004: val_loss did not improve from 0.21336
70/70 - 691s - loss: 0.2536 - accuracy: 0.9700 - val_loss: 0.2925 - val_accuracy: 0.9660
Epoch 5/10

Epoch 00005: val_loss improved from 0.21336 to 0.14208, saving model to mymodel.h5
70/70 - 469s - loss: 0.1367 - accuracy: 0.9843 - val_loss: 0.1421 - val_accuracy: 0.9839
Epoch 6/10

Epoch 00006: val_loss did no